# AI Studio Azure batch run Evaluation
### Intent Prompt Flow - Base Run

Now in order to test these more thoroughly, we can use the Azure AI Studio to run batches of test data with the evaluation prompt flow on a larger dataset.

In [1]:
import json
# Import required libraries
from promptflow.azure import PFClient
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from evaluate import run_azure_flow, run_azure_eval_flow

In [2]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

Populate the `config.json` file with the subscription_id, resource_group, and workspace_name.

In [3]:
config_path = "../config.json"
pf_azure_client = PFClient.from_config(credential=credential, path=config_path)

Found the config file in: ../config.json


Set the properties needed to run in Azure

In [4]:
# Update the runtime to the name of the runtime you created previously
runtime = "automatic"
flow = "../contoso-intent"
data = "../data/alltestdata.jsonl"
run_name = "intent_base_run"
column_mapping={"customerId": "${data.customerId}","question": "${data.question}"}


Create a base run to use as the variant for the evaluation runs. 

In [5]:
base_run = run_azure_flow(runtime, flow, run_name, data, column_mapping, pf_azure_client)

Uploading contoso-intent (0.08 MBs): 100%|██████████| 80322/80322 [00:01<00:00, 40548.63it/s]


[2024-03-05 19:35:11,490][promptflow][WARNING] - You're using automatic runtime, if it's first time you're using it, it may take a while to build runtime and you may see 'NotStarted' status for a while. 


Portal url: https://ai.azure.com/projectflows/bulkrun/run/intent_base_run_03_05_1935/details?wsid=/subscriptions/a195fdab-ef20-44e0-ae3a-48b2e1764e85/resourcegroups/contchat-rg/providers/Microsoft.MachineLearningServices/workspaces/contoso-chat-sf-aiproj


In [6]:
pf_azure_client.stream(base_run)

(Run status is 'NotStarted', continue streaming...)
======= Run Summary =======
Run name: "intent_base_run_03_05_1935"
Run status: "Failed"
Start time: "2024-03-05 11:36:15.618502+00:00"
Duration: "0:00:00"
Run url: "https://ai.azure.com/projectflows/bulkrun/run/intent_base_run_03_05_1935/details?wsid=/subscriptions/a195fdab-ef20-44e0-ae3a-48b2e1764e85/resourcegroups/contchat-rg/providers/Microsoft.MachineLearningServices/workspaces/contoso-chat-sf-aiproj"

InvalidRunStatusError: Failed to submit bulk run due to Not found connection contoso-support,contoso-chat-model for user Sunil Sattiraju

In [ ]:
details = pf_azure_client.get_details(base_run)
details.head(10)

## Intent Prompt Flow Evaluation - Eval Run

In [ ]:
eval_flow = "multi_flow/"
data = "../data/alltestdata.jsonl"
run_name = "intent_eval_run"
column_mapping={
        # reference data
        "customerId": "${data.customerId}",
        "question": "${data.question}",
        "context": "${run.outputs.context}",
        # reference the run's output
        "answer": "${run.outputs.answer}",
    }

In [ ]:
eval_run = run_azure_eval_flow(runtime, eval_flow, run_name, data, column_mapping, base_run, pf_azure_client)

In [ ]:
pf_azure_client.stream(eval_run)

In [ ]:
details = pf_azure_client.get_details(eval_run)
details.head(10)

In [ ]:

metrics = pf_azure_client.get_metrics(eval_run)
print(json.dumps(metrics, indent=4))

In [ ]:
pf_azure_client.visualize([base_run, eval_run])